# NEO4J DESKTOP --VERSIONE CON APPROSSIMAZIONE

ASSUMIAMO CHE SHAPE SIA UNA PROPRIETA DELLA RELAZIONE TRA DUE NODI

In [ ]:
# CELLA 1: IMPORTAZIONI
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import rasterio
from neo4j import GraphDatabase
import time
import os

print("Librerie caricate correttamente.")

In [ ]:
# CELLA 2: CONFIGURAZIONE LOCALE
# --- CONFIGURAZIONE NEO4J DESKTOP ---
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "LA_TUA_PASSWORD_LOCALE" # <--- Inserisci la tua password qui

# --- FILE LOCALI ---
# IMPORTANTE: I file devono trovarsi nella STESSA CARTELLA di questo notebook
# --- FILE LOCALI ---
DATA_PATH= '/content/data/'
FILES = {
    'agency': DATA_PATH +'agency.txt',
    'routes': DATA_PATH +'routes.txt',
    'trips': DATA_PATH+ 'trips.txt',
    'stops': DATA_PATH + 'stops.txt',
    'stop_times': DATA_PATH + 'stop_times.txt',
    'shapes': DATA_PATH + 'shapes.txt',
    'calendar': DATA_PATH + 'new_calendar_dates.txt',
    'geojson': DATA_PATH+ 'QuartieriModena.geojson'
}




RASTER_FILES = {
    'morning': DATA_PATH +'crash_risk_morning.tif',
    'afternoon': DATA_PATH +'crash_risk_afternoon_raster.tif',
    'night': DATA_PATH + 'crash_risk_night_raster.tif'
}

# --- TEST CONNESSIONE ---
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

def verify_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Connessione Locale OK' AS status")
            print(result.single()['status'])
    except Exception as e:
        print(f"ERRORE DI CONNESSIONE: {e}")
        print("Assicurati che il database in Neo4j Desktop sia avviato (Start).")

verify_connection(driver)

In [ ]:
# CELLA 3: HELPER FUNCTIONS
def load_dataframe_to_neo4j(session, query, df, batch_size=2000, **kwargs):
    """Carica DataFrame in Neo4j a blocchi."""
    total = len(df)
    if total == 0:
        print("   Warning: DataFrame vuoto, nulla da caricare.")
        return

    print(f"   Caricamento {total} righe in Neo4j...")
    # Converti NaN in None per Cypher e forza conversione tipi base
    data = df.where(pd.notnull(df), None).to_dict('records')

    start = time.time()
    for i in range(0, total, batch_size):
        batch = data[i:i+batch_size]
        session.run(query, batch=batch, **kwargs)
        print(f"   Progresso: {min(i+batch_size, total)}/{total}", end='\r')

    print(f"\n   Fatto in {time.time() - start:.2f}s.")

In [ ]:
# CELLA 4: CARICAMENTO E FILTRAGGIO DATI (SOLO URBANI)
def load_and_filter_data():
    print("Caricamento CSV e Filtraggio...")

    # 1. ROUTES: Filtra per lista specifica
    routes = pd.read_csv(FILES['routes'], dtype=str)

    # Lista linee urbane Modena
    TARGET_LINES = ['1', '2', '3', '4', '5', '5taxi', '6', '7', '7A', '8', '9', '10', '10taxi', '11', '12', '13']

    # Pulizia stringhe
    routes['route_short_name'] = routes['route_short_name'].astype(str).str.strip()

    # Filtro
    urban_routes = routes[routes['route_short_name'].isin(TARGET_LINES)].copy()

    print(f"   Routes Totali: {len(routes)} -> Selezionate: {len(urban_routes)}")
    if len(urban_routes) == 0:
        print("ERRORE: Nessuna linea trovata! Verifica i nomi nel file routes.txt.")
        return None, None, None, None, None

    valid_route_ids = set(urban_routes['route_id'])

    # 2. TRIPS: Solo quelli delle rotte selezionate
    trips = pd.read_csv(FILES['trips'], dtype=str)
    trips = trips[trips['route_id'].isin(valid_route_ids)]
    print(f"   Trips mantenuti: {len(trips)}")
    valid_trip_ids = set(trips['trip_id'])

    # 3. STOP_TIMES: Solo quelli dei trip selezionati
    stoptimes = pd.read_csv(FILES['stop_times'], dtype={'trip_id': str, 'stop_id': str, 'stop_sequence': int})
    stoptimes = stoptimes[stoptimes['trip_id'].isin(valid_trip_ids)]
    stoptimes = stoptimes.sort_values(by=['trip_id', 'stop_sequence']) # Importante ordinare
    print(f"   StopTimes mantenuti: {len(stoptimes)}")

    # 4. STOPS: Solo quelle usate
    valid_stop_ids = set(stoptimes['stop_id'])
    stops = pd.read_csv(FILES['stops'], dtype={'stop_id': str})
    stops = stops[stops['stop_id'].isin(valid_stop_ids)]
    print(f"   Stops mantenute: {len(stops)}")

    # 5. SHAPES: Solo quelle usate dai trip
    valid_shape_ids = set(trips['shape_id'].dropna())
    shapes = pd.read_csv(FILES['shapes'], dtype={'shape_id': str})
    shapes = shapes[shapes['shape_id'].isin(valid_shape_ids)]
    shapes = shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])
    print(f"   Punti Shape mantenuti: {len(shapes)}")

    return urban_routes, trips, stoptimes, stops, shapes

# Esegui
df_routes, df_trips, df_st, df_stops, df_shapes = load_and_filter_data()

In [ ]:
# CELLA 5: ELABORAZIONE SHAPES E RISCHIO (IN MEMORIA)
def process_shapes_and_risks(df_shapes, df_st, df_stops, raster_files):
    if df_shapes is None or len(df_shapes) == 0:
        print("Nessuna shape da elaborare.")
        return pd.DataFrame()

    print("Apertura Raster e campionamento punti...")

    # 1. Apri Raster
    datasets = {}
    for k, v in raster_files.items():
        try:
            datasets[k] = rasterio.open(v)
        except:
            print(f"Warning: Impossibile aprire {v}")

    # 2. Arricchisci Shapes con rischio (Loop sui punti)
    # Per velocizzare, creiamo colonne numpy
    lons = df_shapes['shape_pt_lon'].values
    lats = df_shapes['shape_pt_lat'].values

    for k, ds in datasets.items():
        risk_vals = []
        # Lettura raster (può essere lenta, ma su dati filtrati è ok)
        for lon, lat in zip(lons, lats):
            try:
                r, c = ds.index(lon, lat)
                val = ds.read(1)[r, c]
                risk_vals.append(float(val) if val >= 0 else 0.0)
            except:
                risk_vals.append(0.0)
        df_shapes[f'risk_{k}'] = risk_vals

    # Chiudi raster
    for ds in datasets.values(): ds.close()

    print("Creazione segmenti Stop-to-Stop...")

    # Lookup veloci
    shapes_dict = {k: v for k, v in df_shapes.groupby('shape_id')}
    stops_loc = df_stops.set_index('stop_id')[['stop_lat', 'stop_lon']].to_dict('index')

    precedes_data = []
    grouped_st = df_st.groupby('trip_id')

    # Trip Lookup
    trip_shape_map = df_trips.set_index('trip_id')['shape_id'].to_dict()

    count = 0
    total = len(grouped_st)

    for trip_id, group in grouped_st:
        count += 1
        if count % 200 == 0: print(f"   Elaborati {count}/{total} trips...", end='\r')

        shape_id = trip_shape_map.get(trip_id)
        stops_seq = group.to_dict('records')

        # Itera coppie di stop (A -> B)
        for i in range(len(stops_seq) - 1):
            st_A = stops_seq[i]
            st_B = stops_seq[i+1]

            props = {
                'trip_id': trip_id,
                'stop_seq_A': st_A['stop_sequence'],
                'departure_time': st_A['departure_time'],
                'arrival_time': st_B['arrival_time'],
                'geometry': None,
                'avg_risk_morning': 0.0, 'avg_risk_afternoon': 0.0,
                'avg_risk_night': 0.0
            }

            # Se abbiamo la shape, tagliamo il segmento
            if shape_id and shape_id in shapes_dict:
                pts = shapes_dict[shape_id]
                latA, lonA = stops_loc[st_A['stop_id']]['stop_lat'], stops_loc[st_A['stop_id']]['stop_lon']
                latB, lonB = stops_loc[st_B['stop_id']]['stop_lat'], stops_loc[st_B['stop_id']]['stop_lon']

                # Trova indici più vicini (Euclidea quadrata)
                distsA = (pts['shape_pt_lat'] - latA)**2 + (pts['shape_pt_lon'] - lonA)**2
                distsB = (pts['shape_pt_lat'] - latB)**2 + (pts['shape_pt_lon'] - lonB)**2

                idx_A = distsA.idxmin()
                idx_B = distsB.idxmin()

                # Gestione ordine (se idx_A > idx_B c'è qualcosa di strano, ma prendiamo il range comunque)
                start, end = min(idx_A, idx_B), max(idx_A, idx_B)

                # Estrai segmento
                segment = pts.loc[start:end]

                if not segment.empty:
                    # Calcola medie rischi
                    for t in ['morning', 'afternoon', 'night']:
                        props[f'avg_risk_{t}'] = segment[f'risk_{t}'].mean()

                    # Geometria WKT
                    coords = list(zip(segment['shape_pt_lon'], segment['shape_pt_lat']))
                    if len(coords) < 2: coords.append(coords[0]) # Evita punti singoli
                    props['geometry'] = str(LineString(coords))

            precedes_data.append(props)

    print("\nElaborazione completata.")
    return pd.DataFrame(precedes_data)

# Esegui
df_precedes_enriched = process_shapes_and_risks(df_shapes, df_st, df_stops, RASTER_FILES)

In [ ]:
# CELLA 6: SETUP DB
def setup_db(driver):
    with driver.session() as session:
        # PULIZIA TOTALE (Decommenta se vuoi resettare il DB ogni volta)
        # print("Pulizia Database...")
        session.run("MATCH (n) DETACH DELETE n")

        print("Creazione Vincoli...")
        cmds = [
            "CREATE CONSTRAINT agency_id IF NOT EXISTS FOR (a:Agency) REQUIRE a.id IS UNIQUE",
            "CREATE CONSTRAINT route_id IF NOT EXISTS FOR (r:Route) REQUIRE r.id IS UNIQUE",
            "CREATE CONSTRAINT trip_id IF NOT EXISTS FOR (t:Trip) REQUIRE t.id IS UNIQUE",
            "CREATE CONSTRAINT stop_id IF NOT EXISTS FOR (s:Stop) REQUIRE s.id IS UNIQUE",
            "CREATE CONSTRAINT stoptime_id IF NOT EXISTS FOR (st:Stoptime) REQUIRE (st.trip_id, st.stop_sequence) IS UNIQUE"
        ]
        for c in cmds: session.run(c)
        print("DB Pronto.")

setup_db(driver)

In [ ]:
# CELLA 7: CARICAMENTO DATI IN NEO4J
def upload_graph(driver, df_routes, df_trips, df_stops, df_st, df_precedes_enriched):
    with driver.session() as session:
        # 1. Agency (Default ID '1' se manca)
        df_agency = pd.read_csv(FILES['agency'], dtype=str)
        if 'agency_id' not in df_agency.columns: df_agency['agency_id'] = '1'
        q_ag = """UNWIND $batch AS row MERGE (a:Agency {id: row.agency_id}) SET a.name = row.agency_name"""
        load_dataframe_to_neo4j(session, q_ag, df_agency)

        # 2. Routes
        q_routes = """
        UNWIND $batch as row
        MATCH (a:Agency) LIMIT 1
        MERGE (r:Route {id: row.route_id})
        SET r.short_name = row.route_short_name, r.long_name = row.route_long_name
        MERGE (a)-[:OPERATES]->(r)
        """
        load_dataframe_to_neo4j(session, q_routes, df_routes)

        # 3. Trips
        q_trips = """
        UNWIND $batch as row
        MATCH (r:Route {id: row.route_id})
        MERGE (t:Trip {id: row.trip_id})
        SET t.headsign = row.trip_headsign, t.service_id = row.service_id
        MERGE (r)<-[:USES]-(t)
        """
        load_dataframe_to_neo4j(session, q_trips, df_trips)

        # 4. Stops
        q_stops = """
        UNWIND $batch as row
        MERGE (s:Stop {id: row.stop_id})
        SET s.name = row.stop_name, s.lat = toFloat(row.stop_lat), s.lon = toFloat(row.stop_lon)
        """
        load_dataframe_to_neo4j(session, q_stops, df_stops)

        # 5. StopTimes (Nodi)
        print("Caricamento nodi Stoptime...")
        q_st = """
        UNWIND $batch as row
        MATCH (t:Trip {id: row.trip_id}), (s:Stop {id: row.stop_id})
        MERGE (st:Stoptime {trip_id: row.trip_id, stop_sequence: toInteger(row.stop_sequence)})
        SET st.arrival_time = time(row.arrival_time), st.departure_time = time(row.departure_time)
        MERGE (t)<-[:PART_OF_TRIP]-(st)
        MERGE (st)-[:LOCATED_AT]->(s)
        """
        load_dataframe_to_neo4j(session, q_st, df_st)

        # 6. RELAZIONE PRECEDES (con Geometria e Rischio)
        print("Caricamento relazioni PRECEDES arricchite...")
        q_precedes = """
        UNWIND $batch as row
        MATCH (stA:Stoptime {trip_id: row.trip_id, stop_sequence: toInteger(row.stop_seq_A)})
        MATCH (stB:Stoptime {trip_id: row.trip_id, stop_sequence: toInteger(row.stop_seq_A) + 1})
        MERGE (stA)-[p:PRECEDES]->(stB)
        SET p.geometry = row.geometry,
            p.waiting_time = duration.inSeconds(time(row.departure_time), time(row.arrival_time)).seconds,
            p.risk_morning = toFloat(row.avg_risk_morning),
            p.risk_afternoon = toFloat(row.avg_risk_afternoon),
            p.risk_evening = toFloat(row.avg_risk_evening),
            p.risk_night = toFloat(row.avg_risk_night)
        """
        load_dataframe_to_neo4j(session, q_precedes, df_precedes_enriched)

        # 7. WALK_TO
        print("Creazione WALK_TO (< 300m)...")
        session.run("""
            MATCH (s1:Stop), (s2:Stop)
            WHERE elementId(s1) < elementId(s2)
            WITH s1, s2, point.distance(point({latitude: s1.lat, longitude: s1.lon}),
                                        point({latitude: s2.lat, longitude: s2.lon})) AS dist
            WHERE dist < 300
            MERGE (s1)-[:WALK_TO {distance: dist}]->(s2)
            MERGE (s2)-[:WALK_TO {distance: dist}]->(s1)
        """)

upload_graph(driver, df_routes, df_trips, df_stops, df_st, df_precedes_enriched)

In [ ]:
# CELLA 8: CARICAMENTO QUARTIERI
def load_neighborhoods(driver):
    print("Caricamento GeoJSON Quartieri...")
    gdf = gpd.read_file(FILES['geojson']).to_crs("EPSG:4326")

    neighborhoods = []
    for idx, row in gdf.iterrows():
        centroid = row.geometry.centroid
        name = row.get('nome', f"Q_{idx}")
        neighborhoods.append({'name': name, 'lat': centroid.y, 'lon': centroid.x})

    with driver.session() as session:
        q = """
        UNWIND $batch as row
        MERGE (n:Neighborhood {name: row.name})
        SET n.lat = row.lat, n.lon = row.lon
        """
        load_dataframe_to_neo4j(session, q, pd.DataFrame(neighborhoods))

        # Collegamento Access Point
        session.run("""
            MATCH (n:Neighborhood)
            WITH n, point({latitude: n.lat, longitude: n.lon}) as pN
            MATCH (s:Stop)
            WITH n, s, point.distance(pN, point({latitude: s.lat, longitude: s.lon})) as dist
            ORDER BY dist ASC
            WITH n, collect(s)[0] as closest, dist
            MERGE (n)-[:ACCESS_POINT {distance: dist}]->(closest)
        """)
    print("Quartieri collegati.")

load_neighborhoods(driver)

In [ ]:
# CELLA 9: ROUTING DIJKSTRA (TIME + RISK)
def run_routing(driver, time_of_day='morning', risk_penalty=1000):
    print(f"--- Calcolo Percorsi: {time_of_day.upper()} (Penalità Rischio: {risk_penalty}) ---")

    # Nome proprietà rischio
    risk_prop = f"risk_{time_of_day}"

    # Formula Peso: secondi + (probabilità * penalità)
    # Usa coalesce per gestire eventuali nulli come rischio 0
    weight_q = f"waiting_time + (coalesce({risk_prop}, 0.0) * {risk_penalty})"

    graph_name = f"urban_simpler_{time_of_day}"

    with driver.session() as session:
        # 1. Drop proiezione precedente
        session.run(f"CALL gds.graph.drop('{graph_name}', false)")

        # 2. Proiezione Grafo
        print("Proiezione Grafo in memoria...")
        session.run(f"""
            CALL gds.graph.project(
                '{graph_name}',
                ['Neighborhood', 'Stop', 'Stoptime'],
                {{
                    PRECEDES: {{
                        type: 'PRECEDES',
                        properties: {{ weight: '{weight_q}' }}
                    }},
                    WALK_TO: {{
                        type: 'WALK_TO',
                        properties: {{ weight: 'distance / 1.3' }}
                    }},
                    ACCESS_POINT: {{
                        type: 'ACCESS_POINT',
                        orientation: 'UNDIRECTED',
                        properties: {{ weight: 'distance / 1.3' }}
                    }},
                    LOCATED_AT: {{ type: 'LOCATED_AT', orientation: 'UNDIRECTED' }},
                    PART_OF_TRIP: {{ type: 'PART_OF_TRIP', orientation: 'UNDIRECTED' }}
                }}
            )
        """)

        # 3. Esecuzione Dijkstra
        print("Calcolo Shortest Path...")
        result = session.run(f"""
            MATCH (n1:Neighborhood), (n2:Neighborhood)
            WHERE elementId(n1) < elementId(n2)

            CALL gds.shortestPath.dijkstra.stream('{graph_name}', {{
                sourceNode: n1,
                targetNode: n2,
                relationshipWeightProperty: 'weight'
            }})
            YIELD totalCost

            RETURN n1.name as Da, n2.name as A, totalCost as Score_Totale
            LIMIT 20
        """)

        df_res = pd.DataFrame([r.data() for r in result])

        # Cleanup
        session.run(f"CALL gds.graph.drop('{graph_name}', false)")
        return df_res

# Esegui analisi
df_results = run_routing(driver, 'morning', risk_penalty=1000)
print(df_results)

# NEO4J DESKTOP-- VERSIONE CHE CREA SHAPE POINT

In [ ]:
# CELLA 1: IMPORTAZIONI
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from neo4j import GraphDatabase
import time
import os

print("Librerie caricate correttamente.")

In [ ]:
# CELLA 2: CONFIGURAZIONE LOCALE
# --- CONFIGURAZIONE NEO4J DESKTOP ---
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "LA_TUA_PASSWORD_LOCALE" # <--- Inserisci la tua password qui

# --- FILE LOCALI ---
# IMPORTANTE: I file devono trovarsi nella STESSA CARTELLA di questo notebook
# --- FILE LOCALI ---
DATA_PATH= '/content/data/'
FILES = {
    'agency': DATA_PATH +'agency.txt',
    'routes': DATA_PATH +'routes.txt',
    'trips': DATA_PATH+ 'trips.txt',
    'stops': DATA_PATH + 'stops.txt',
    'stop_times': DATA_PATH + 'stop_times.txt',
    'shapes': DATA_PATH + 'shapes.txt',
    'calendar': DATA_PATH + 'new_calendar_dates.txt',
    'geojson': DATA_PATH+ 'QuartieriModena.geojson'
}




RASTER_FILES = {
    'morning': DATA_PATH +'crash_risk_morning.tif',
    'afternoon': DATA_PATH +'crash_risk_afternoon_raster.tif',
    'night': DATA_PATH + 'crash_risk_night_raster.tif'
}

# --- TEST CONNESSIONE ---
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

def verify_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Connessione Locale OK' AS status")
            print(result.single()['status'])
    except Exception as e:
        print(f"ERRORE DI CONNESSIONE: {e}")
        print("Assicurati che il database in Neo4j Desktop sia avviato (Start).")

verify_connection(driver)

In [ ]:
# CELLA 3: HELPER LOAD BATCH
def load_dataframe_to_neo4j(session, query, df, batch_size=2000, **kwargs):
    """Carica DataFrame in Neo4j a blocchi."""
    total = len(df)
    if total == 0: return

    print(f"   Caricamento {total} righe...")
    # Gestione NaN -> None per Cypher
    data = df.where(pd.notnull(df), None).to_dict('records')

    start = time.time()
    for i in range(0, total, batch_size):
        batch = data[i:i+batch_size]
        session.run(query, batch=batch, **kwargs)
        print(f"   Progresso: {min(i+batch_size, total)}/{total}", end='\r')

    print(f"\n   Completato in {time.time() - start:.2f}s.")

In [ ]:
# CELLA 4: FILTRAGGIO DATI (SOLO LINEE URBANE)
def load_and_filter_data():
    print("Caricamento e Filtraggio Dati...")

    # 1. ROUTES
    routes = pd.read_csv(FILES['routes'], dtype=str)

    # TUE LINEE URBANE
    TARGET_LINES = ['1', '2', '3', '4', '5', '5taxi', '6', '7', '7A', '8', '9', '10', '10taxi', '11', '12', '13']

    # Pulizia e Filtro
    routes['route_short_name'] = routes['route_short_name'].astype(str).str.strip()
    urban_routes = routes[routes['route_short_name'].isin(TARGET_LINES)].copy()

    print(f"   Routes: {len(routes)} -> {len(urban_routes)} (Urbane)")

    if len(urban_routes) == 0:
        print("ERRORE: Nessuna linea trovata. Controlla i nomi in routes.txt")
        return None, None, None, None, None

    valid_route_ids = set(urban_routes['route_id'])

    # 2. TRIPS
    trips = pd.read_csv(FILES['trips'], dtype=str)
    trips = trips[trips['route_id'].isin(valid_route_ids)]
    print(f"   Trips: {len(trips)}")
    valid_trip_ids = set(trips['trip_id'])

    # 3. STOP_TIMES
    stoptimes = pd.read_csv(FILES['stop_times'], dtype={'trip_id': str, 'stop_id': str, 'stop_sequence': int})
    stoptimes = stoptimes[stoptimes['trip_id'].isin(valid_trip_ids)]
    print(f"   StopTimes: {len(stoptimes)}")

    # 4. STOPS
    valid_stop_ids = set(stoptimes['stop_id'])
    stops = pd.read_csv(FILES['stops'], dtype={'stop_id': str})
    stops = stops[stops['stop_id'].isin(valid_stop_ids)]
    print(f"   Stops: {len(stops)}")

    # 5. SHAPES (PUNTI GEOMETRICI)
    # Teniamo solo i punti delle shape usate dai trip urbani
    valid_shape_ids = set(trips['shape_id'].dropna())
    shapes = pd.read_csv(FILES['shapes'], dtype={'shape_id': str})
    shapes = shapes[shapes['shape_id'].isin(valid_shape_ids)]
    # Ordine essenziale per la sequenza
    shapes = shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])
    print(f"   ShapePoints: {len(shapes)}")

    return urban_routes, trips, stoptimes, stops, shapes

# Esegui
df_routes, df_trips, df_st, df_stops, df_shapes = load_and_filter_data()

In [ ]:
# CELLA 5: CARICAMENTO GRAFO COMPLETO (NODES + SHAPEPOINTS)
def upload_full_graph(driver, df_routes, df_trips, df_stops, df_shapes):
    with driver.session() as session:
        # Pulizia opzionale (Decommenta per resettare)
        # session.run("MATCH (n) DETACH DELETE n")

        # Vincoli
        print("Creazione Vincoli...")
        cmds = [
            "CREATE CONSTRAINT agency_id IF NOT EXISTS FOR (a:Agency) REQUIRE a.id IS UNIQUE",
            "CREATE CONSTRAINT route_id IF NOT EXISTS FOR (r:Route) REQUIRE r.id IS UNIQUE",
            "CREATE CONSTRAINT trip_id IF NOT EXISTS FOR (t:Trip) REQUIRE t.id IS UNIQUE",
            "CREATE CONSTRAINT stop_id IF NOT EXISTS FOR (s:Stop) REQUIRE s.id IS UNIQUE",
            # Vincolo composto per ShapePoints (ShapeID + Sequenza)
            "CREATE CONSTRAINT shape_pt_id IF NOT EXISTS FOR (sp:ShapePoint) REQUIRE (sp.shape_id, sp.sequence) IS UNIQUE"
        ]
        for c in cmds: session.run(c)

        # 1. Agency
        print("1. Agency...")
        df_ag = pd.read_csv(FILES['agency'], dtype=str)
        if 'agency_id' not in df_ag.columns: df_ag['agency_id'] = '1'
        q_ag = "UNWIND $batch as row MERGE (a:Agency {id: row.agency_id}) SET a.name = row.agency_name"
        load_dataframe_to_neo4j(session, q_ag, df_ag)

        # 2. Routes
        print("2. Routes...")
        q_r = """
        UNWIND $batch as row
        MATCH (a:Agency) LIMIT 1
        MERGE (r:Route {id: row.route_id})
        SET r.short_name = row.route_short_name, r.long_name = row.route_long_name
        MERGE (a)-[:OPERATES]->(r)
        """
        load_dataframe_to_neo4j(session, q_r, df_routes)

        # 3. Trips
        print("3. Trips...")
        q_t = """
        UNWIND $batch as row
        MATCH (r:Route {id: row.route_id})
        MERGE (t:Trip {id: row.trip_id})
        SET t.headsign = row.trip_headsign, t.service_id = row.service_id, t.shape_id = row.shape_id
        MERGE (r)<-[:USES]-(t)
        """
        load_dataframe_to_neo4j(session, q_t, df_trips)

        # 4. Stops
        print("4. Stops...")
        q_s = """
        UNWIND $batch as row
        MERGE (s:Stop {id: row.stop_id})
        SET s.lat = toFloat(row.stop_lat), s.lon = toFloat(row.stop_lon), s.name = row.stop_name
        """
        load_dataframe_to_neo4j(session, q_s, df_stops)

        # 5. SHAPE POINTS (Il cuore di questa versione)
        print("5. ShapePoints (Nodi geometrici)...")
        q_sp = """
        UNWIND $batch as row
        MERGE (sp:ShapePoint {shape_id: row.shape_id, sequence: toInteger(row.shape_pt_sequence)})
        SET sp.lat = toFloat(row.shape_pt_lat), sp.lon = toFloat(row.shape_pt_lon)
        """
        load_dataframe_to_neo4j(session, q_sp, df_shapes, batch_size=5000)

        print("   Collegamento ShapePoints (FOLLOWS)...")
        # Usiamo una query Cypher ottimizzata per collegare la sequenza
        # Poiché abbiamo filtrato i dati, questa operazione su Desktop è veloce
        session.run("""
            CALL apoc.periodic.iterate(
                "MATCH (sp:ShapePoint) RETURN sp ORDER BY sp.shape_id, sp.sequence",
                "WITH sp
                 MATCH (prev:ShapePoint {shape_id: sp.shape_id, sequence: sp.sequence - 1})
                 MERGE (prev)-[r:FOLLOWS]->(sp)
                 SET r.distance = point.distance(point({latitude: prev.lat, longitude: prev.lon}),
                                                 point({latitude: sp.lat, longitude: sp.lon}))",
                {batchSize: 2000, parallel: false}
            )
        """)
        print("Grafo geometrico costruito.")

upload_full_graph(driver, df_routes, df_trips, df_stops, df_shapes)

In [ ]:
# CELLA 6: STOPTIMES E WALK
def load_schedule(driver, df_st):
    with driver.session() as session:
        print("Caricamento StopTimes...")
        q_st = """
        UNWIND $batch as row
        MATCH (t:Trip {id: row.trip_id}), (s:Stop {id: row.stop_id})
        MERGE (st:Stoptime {trip_id: row.trip_id, stop_sequence: toInteger(row.stop_sequence)})
        SET st.arrival_time = time(row.arrival_time), st.departure_time = time(row.departure_time)
        MERGE (t)<-[:PART_OF_TRIP]-(st)
        MERGE (st)-[:LOCATED_AT]->(s)
        """
        load_dataframe_to_neo4j(session, q_st, df_st)

        print("Creazione Relazioni PRECEDES (Base)...")
        session.run("""
            CALL apoc.periodic.iterate(
                "MATCH (t:Trip) RETURN t",
                "MATCH (t)<-[:PART_OF_TRIP]-(st1:Stoptime)
                 MATCH (t)<-[:PART_OF_TRIP]-(st2:Stoptime)
                 WHERE st2.stop_sequence = st1.stop_sequence + 1
                 MERGE (st1)-[p:PRECEDES]->(st2)
                 SET p.waiting_time = duration.inSeconds(st1.departure_time, st2.arrival_time).seconds",
                {batchSize: 500}
            )
        """)

        print("Creazione WALK_TO (< 300m)...")
        session.run("""
            MATCH (s1:Stop), (s2:Stop)
            WHERE elementId(s1) < elementId(s2)
            WITH s1, s2, point.distance(point({latitude: s1.lat, longitude: s1.lon}),
                                        point({latitude: s2.lat, longitude: s2.lon})) AS dist
            WHERE dist < 300
            MERGE (s1)-[:WALK_TO {distance: dist}]->(s2)
            MERGE (s2)-[:WALK_TO {distance: dist}]->(s1)
        """)

load_schedule(driver, df_st)

In [ ]:
# CELLA 7: INTEGRAZIONE RASTER (SU SHAPE POINTS)
def integrate_risk_on_shapes(driver, raster_files):
    print("Inizio calcolo rischi sui segmenti stradali...")

    # 1. Apri Raster
    datasets = {}
    for k, v in raster_files.items():
        try:
            datasets[k] = rasterio.open(v)
        except:
            print(f"Warning: {v} non trovato.")

    # 2. Recupera segmenti fisici
    print("Recupero segmenti FOLLOWS...")
    with driver.session() as session:
        res = session.run("""
            MATCH (s1:ShapePoint)-[r:FOLLOWS]->(s2:ShapePoint)
            RETURN elementId(r) as id, s1.lat as lat1, s1.lon as lon1, s2.lat as lat2, s2.lon as lon2
        """)
        segments = [r.data() for r in res]

    print(f"Elaborazione di {len(segments)} segmenti...")
    updates = []

    for seg in segments:
        mid_lat = (seg['lat1'] + seg['lat2']) / 2
        mid_lon = (seg['lon1'] + seg['lon2']) / 2

        props = {'id': seg['id']}
        for k, ds in datasets.items():
            try:
                r, c = ds.index(mid_lon, mid_lat)
                val = ds.read(1)[r, c]
                props[f'risk_{k}'] = float(val) if val >= 0 else 0.0
            except:
                props[f'risk_{k}'] = 0.0
        updates.append(props)

    # 3. Aggiorna Neo4j
    print("Salvataggio Rischi nel DB...")
    with driver.session() as session:
        q_upd = """
        UNWIND $batch as row
        MATCH ()-[r:FOLLOWS]->() WHERE elementId(r) = row.id
        SET r.risk_morning = row.risk_morning,
            r.risk_afternoon = row.risk_afternoon,
            r.risk_evening = row.risk_evening,
            r.risk_night = row.risk_night
        """
        load_dataframe_to_neo4j(session, q_upd, pd.DataFrame(updates), batch_size=5000)

    for ds in datasets.values(): ds.close()
    print("Integrazione Raster completata.")

integrate_risk_on_shapes(driver, RASTER_FILES)

In [ ]:
# CELLA 8: PROPAGAZIONE RISCHIO
def propagate_risk(driver):
    print("Propagazione Rischio (Shape -> Trip)...")

    # Calcola media rischio per ShapeID e aggiorna i Trip
    query = """
    CALL apoc.periodic.iterate(
        "MATCH (sp1:ShapePoint)-[r:FOLLOWS]->(sp2:ShapePoint)
         RETURN sp1.shape_id AS shape_id,
                avg(r.risk_morning) as rm, avg(r.risk_afternoon) as ra,
                avg(r.risk_evening) as re, avg(r.risk_night) as rn",
        "MATCH (t:Trip {shape_id: shape_id})
         MATCH (t)<-[:PART_OF_TRIP]-(st1:Stoptime)-[p:PRECEDES]->(st2:Stoptime)
         SET p.risk_morning = rm,
             p.risk_afternoon = ra,
             p.risk_evening = re,
             p.risk_night = rn",
        {batchSize: 100, parallel: false}
    )
    """
    with driver.session() as session:
        session.run(query)
    print("Propagazione completata.")

propagate_risk(driver)

In [ ]:
# CELLA 9: CARICAMENTO QUARTIERI
def load_neighborhoods(driver):
    print("Caricamento Quartieri...")
    gdf = gpd.read_file(FILES['geojson']).to_crs("EPSG:4326")

    neighborhoods = []
    for idx, row in gdf.iterrows():
        centroid = row.geometry.centroid
        neighborhoods.append({'name': row.get('nome', f"Q_{idx}"), 'lat': centroid.y, 'lon': centroid.x})

    with driver.session() as session:
        q = "UNWIND $batch as row MERGE (n:Neighborhood {name: row.name}) SET n.lat = row.lat, n.lon = row.lon"
        load_dataframe_to_neo4j(session, q, pd.DataFrame(neighborhoods))

        session.run("""
            MATCH (n:Neighborhood)
            WITH n, point({latitude: n.lat, longitude: n.lon}) as pN
            MATCH (s:Stop)
            WITH n, s, point.distance(pN, point({latitude: s.lat, longitude: s.lon})) as dist
            ORDER BY dist ASC
            WITH n, collect(s)[0] as closest, dist
            MERGE (n)-[:ACCESS_POINT {distance: dist}]->(closest)
        """)
    print("Quartieri pronti.")

load_neighborhoods(driver)

In [ ]:
# CELLA 10: ROUTING DIJKSTRA
def run_routing(driver, time_of_day='morning', risk_penalty=1000):
    print(f"--- Routing: {time_of_day.upper()} (Penalità: {risk_penalty}) ---")

    risk_prop = f"risk_{time_of_day}"
    # Peso = Secondi + (Rischio * Penalità)
    weight_q = f"waiting_time + (coalesce({risk_prop}, 0.0) * {risk_penalty})"
    graph_name = f"full_geo_graph_{time_of_day}"

    with driver.session() as session:
        session.run(f"CALL gds.graph.drop('{graph_name}', false)")

        session.run(f"""
            CALL gds.graph.project(
                '{graph_name}',
                ['Neighborhood', 'Stop', 'Stoptime'],
                {{
                    PRECEDES: {{
                        type: 'PRECEDES',
                        properties: {{ weight: '{weight_q}' }}
                    }},
                    WALK_TO: {{
                        type: 'WALK_TO',
                        properties: {{ weight: 'distance / 1.3' }}
                    }},
                    ACCESS_POINT: {{
                        type: 'ACCESS_POINT',
                        orientation: 'UNDIRECTED',
                        properties: {{ weight: 'distance / 1.3' }}
                    }},
                    LOCATED_AT: {{ type: 'LOCATED_AT', orientation: 'UNDIRECTED' }},
                    PART_OF_TRIP: {{ type: 'PART_OF_TRIP', orientation: 'UNDIRECTED' }}
                }}
            )
        """)

        result = session.run(f"""
            MATCH (n1:Neighborhood), (n2:Neighborhood)
            WHERE elementId(n1) < elementId(n2)
            CALL gds.shortestPath.dijkstra.stream('{graph_name}', {{
                sourceNode: n1, targetNode: n2, relationshipWeightProperty: 'weight'
            }})
            YIELD totalCost
            RETURN n1.name as Da, n2.name as A, totalCost as Score
            LIMIT 20
        """)

        df_res = pd.DataFrame([r.data() for r in result])
        session.run(f"CALL gds.graph.drop('{graph_name}', false)")
        return df_res

# Esegui
print(run_routing(driver, 'morning'))